## Архитектуры свёрточных сетей

In [ ]:
import torch
from torch import nn
import torchvision as tv
from torchsummary import summary
import time

In [ ]:
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0, 0
    net.eval()
    for X, y in data_iter:
        X, y = X.to(device), y.to(device)
        acc_sum += (net(X).argmax(axis=1) == y).sum()
        n += y.shape[0]
    return acc_sum.item() / n

In [ ]:
def train(net, train_iter, test_iter, trainer, num_epochs):
    net.to(device)
    loss = nn.CrossEntropyLoss(reduction='sum')
    net.train()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        
        for i, (X, y) in enumerate(train_iter):
            X, y = X.to(device), y.to(device)
            trainer.zero_grad()
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.shape[0]

            if i % 10 == 0:
              print(f"Step {i}. time since epoch: {time.time() -  start:.3f}. " 
                    f"Train acc: {train_acc_sum / n:.3f}. Train Loss: {train_l_sum / n:.3f}")
        test_acc = evaluate_accuracy(test_iter, net.to(device))
        print('-' * 20)
        print(f'epoch {epoch + 1}, loss {train_l_sum / n:.4f}, train acc {train_acc_sum / n:.3f}'
              f', test acc {test_acc:.3f}, time {time.time() - start:.1f} sec')

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

## DataSet

In [ ]:
BATCH_SIZE = 256
transoforms = tv.transforms.Compose([
    tv.transforms.Resize((224, 224)),
    tv.transforms.ToTensor()
])
train_dataset = tv.datasets.MNIST('.', train=True, transform=transoforms, download=True)
test_dataset = tv.datasets.MNIST('.', train=False, transform=transoforms, download=True)
train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_iter = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

## AlexNet

<img src='https://drive.google.com/uc?id=1bZRmcjqn0BZVBRXCk_OiQRa991_3rPpw' width=300>

In [ ]:
net = nn.Sequential(
    nn.Conv2d(1, 96, kernel_size=11, stride=4),
    nn.ReLU(),
    nn.MaxPool2d(3, stride=2),
    nn.Conv2d(96, 256, kernel_size=5, padding=2),
    nn.ReLU(),
    nn.MaxPool2d(3, stride=2),
    nn.Conv2d(256, 384, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Conv2d(384, 384, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Conv2d(384, 256, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(3, stride=2),
    nn.Flatten(),
    nn.Linear(6400, 4096),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(4096, 4096),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(4096, 10)
)

In [ ]:
summary(net.to(device), input_size=(1, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 54, 54]          11,712
              ReLU-2           [-1, 96, 54, 54]               0
         MaxPool2d-3           [-1, 96, 26, 26]               0
            Conv2d-4          [-1, 256, 26, 26]         614,656
              ReLU-5          [-1, 256, 26, 26]               0
         MaxPool2d-6          [-1, 256, 12, 12]               0
            Conv2d-7          [-1, 384, 12, 12]         885,120
              ReLU-8          [-1, 384, 12, 12]               0
            Conv2d-9          [-1, 384, 12, 12]       1,327,488
             ReLU-10          [-1, 384, 12, 12]               0
           Conv2d-11          [-1, 256, 12, 12]         884,992
             ReLU-12          [-1, 256, 12, 12]               0
        MaxPool2d-13            [-1, 256, 5, 5]               0
          Flatten-14                 [-

In [ ]:
lr, num_epochs  = 0.001, 1
trainer = torch.optim.Adam(net.parameters(), lr=lr)
train(net, train_iter, test_iter, trainer, num_epochs)

Step 0. time since epoch: 0.523. Train acc: 0.125. Train Loss: 2.303
Step 10. time since epoch: 4.459. Train acc: 0.113. Train Loss: 2.340
Step 20. time since epoch: 8.556. Train acc: 0.177. Train Loss: 2.259
Step 30. time since epoch: 12.948. Train acc: 0.291. Train Loss: 1.979
Step 40. time since epoch: 15.751. Train acc: 0.398. Train Loss: 1.706
Step 50. time since epoch: 18.577. Train acc: 0.479. Train Loss: 1.484
Step 60. time since epoch: 21.396. Train acc: 0.543. Train Loss: 1.310
Step 70. time since epoch: 24.197. Train acc: 0.592. Train Loss: 1.170
Step 80. time since epoch: 27.002. Train acc: 0.634. Train Loss: 1.052
Step 90. time since epoch: 29.833. Train acc: 0.667. Train Loss: 0.960
Step 100. time since epoch: 32.833. Train acc: 0.694. Train Loss: 0.884
Step 110. time since epoch: 35.656. Train acc: 0.717. Train Loss: 0.820
Step 120. time since epoch: 38.496. Train acc: 0.736. Train Loss: 0.765
Step 130. time since epoch: 41.317. Train acc: 0.754. Train Loss: 0.716
Step 1

## VGG

<img src='https://drive.google.com/uc?id=1xE_QWszsSEA6fOEcadBsrmHOMZOs7aDr' width=300>

In [ ]:
def vgg_block(num_convs, input_channels, num_channels):
    blk = nn.Sequential(nn.Conv2d(input_channels, num_channels, kernel_size=3, padding=1), nn.ReLU())
    for i in range(num_convs - 1):
        blk.add_module("conv{}".format(i), nn.Conv2d(num_channels, num_channels, kernel_size=3, padding=1))
        blk.add_module("relu{}".format(i), nn.ReLU())
    blk.add_module("pool", nn.MaxPool2d(2, stride=2))
    return blk

In [ ]:
def vgg(conv_arch):
    net = nn.Sequential()

    for i, (num_convs, input_ch, num_channels) in enumerate(conv_arch):
        net.add_module("block{}".format(i), vgg_block(num_convs, input_ch, num_channels))

    
    classifier = nn.Sequential(
        nn.Flatten(),
        
        # nn.Linear(25088, 4096), nn.ReLU(), nn.Dropout(0.5),
        nn.Linear(6272, 4096), nn.ReLU(), nn.Dropout(0.5),

        nn.Linear(4096, 4096), nn.ReLU(), nn.Dropout(0.5),
        nn.Linear(4096, 10))

    net.add_module('classifier', classifier)
    return net

In [ ]:
conv_arch = ((1, 1, 64), (1, 64, 128), (2, 128, 256), (2, 256, 512), (2, 512, 512))

In [ ]:
ratio = 4
small_conv_arch = [(v[0], max(v[1] // ratio, 1), v[2] // ratio) for v in conv_arch]
net = vgg(small_conv_arch)

In [ ]:
summary(net.to(device), input_size=(1, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 224, 224]             160
              ReLU-2         [-1, 16, 224, 224]               0
         MaxPool2d-3         [-1, 16, 112, 112]               0
            Conv2d-4         [-1, 32, 112, 112]           4,640
              ReLU-5         [-1, 32, 112, 112]               0
         MaxPool2d-6           [-1, 32, 56, 56]               0
            Conv2d-7           [-1, 64, 56, 56]          18,496
              ReLU-8           [-1, 64, 56, 56]               0
            Conv2d-9           [-1, 64, 56, 56]          36,928
             ReLU-10           [-1, 64, 56, 56]               0
        MaxPool2d-11           [-1, 64, 28, 28]               0
           Conv2d-12          [-1, 128, 28, 28]          73,856
             ReLU-13          [-1, 128, 28, 28]               0
           Conv2d-14          [-1, 128,

In [ ]:
lr, num_epochs = 0.001, 1
trainer = torch.optim.Adam(net.parameters(), lr=lr)
train(net, train_iter, test_iter, trainer, num_epochs)

Step 0. time since epoch: 0.651. Train acc: 0.113. Train Loss: 2.303
Step 10. time since epoch: 5.161. Train acc: 0.112. Train Loss: 2.303
Step 20. time since epoch: 9.661. Train acc: 0.139. Train Loss: 2.283
Step 30. time since epoch: 14.173. Train acc: 0.246. Train Loss: 2.064
Step 40. time since epoch: 18.685. Train acc: 0.358. Train Loss: 1.765
Step 50. time since epoch: 23.213. Train acc: 0.446. Train Loss: 1.530
Step 60. time since epoch: 27.740. Train acc: 0.516. Train Loss: 1.346
Step 70. time since epoch: 32.278. Train acc: 0.572. Train Loss: 1.194
Step 80. time since epoch: 36.844. Train acc: 0.618. Train Loss: 1.070
Step 90. time since epoch: 41.417. Train acc: 0.655. Train Loss: 0.970
Step 100. time since epoch: 46.017. Train acc: 0.684. Train Loss: 0.889
Step 110. time since epoch: 50.581. Train acc: 0.709. Train Loss: 0.823
Step 120. time since epoch: 55.136. Train acc: 0.729. Train Loss: 0.766
Step 130. time since epoch: 59.668. Train acc: 0.747. Train Loss: 0.718
Step 1

## NiN

<img src='https://drive.google.com/uc?id=1V58tBxw-_dSyGGgdKhe-Um5yYWgrNYBX' width=400>

In [ ]:
def nin_block(input_channels, num_channels, kernel_size, strides, padding):
    blk = nn.Sequential(
          nn.Conv2d(input_channels, num_channels, kernel_size, strides, padding),
          nn.ReLU(),
          nn.Conv2d(num_channels, num_channels, kernel_size=1),
          nn.ReLU(),
          nn.Conv2d(num_channels, num_channels, kernel_size=1),
          nn.ReLU()
    )
    return blk

In [ ]:
net = nn.Sequential(
    nin_block(1, 96, kernel_size=11, strides=4, padding=0),
    nn.MaxPool2d(3, stride=2),
    nin_block(96, 256, kernel_size=5, strides=1, padding=2),
    nn.MaxPool2d(3, stride=2),
    nin_block(256, 384, kernel_size=3, strides=1, padding=1),
    nn.MaxPool2d(3, stride=2),
    nn.Dropout(0.5),
    nin_block(384, 10, kernel_size=3, strides=1, padding=1),
    nn.AvgPool2d(5),
    nn.Flatten()
)

In [ ]:
summary(net.to(device), input_size=(1, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 54, 54]          11,712
              ReLU-2           [-1, 96, 54, 54]               0
            Conv2d-3           [-1, 96, 54, 54]           9,312
              ReLU-4           [-1, 96, 54, 54]               0
            Conv2d-5           [-1, 96, 54, 54]           9,312
              ReLU-6           [-1, 96, 54, 54]               0
         MaxPool2d-7           [-1, 96, 26, 26]               0
            Conv2d-8          [-1, 256, 26, 26]         614,656
              ReLU-9          [-1, 256, 26, 26]               0
           Conv2d-10          [-1, 256, 26, 26]          65,792
             ReLU-11          [-1, 256, 26, 26]               0
           Conv2d-12          [-1, 256, 26, 26]          65,792
             ReLU-13          [-1, 256, 26, 26]               0
        MaxPool2d-14          [-1, 256,

In [ ]:
lr, num_epochs = 0.001, 1
trainer = torch.optim.Adam(net.parameters(), lr=lr)
train(net, train_iter, test_iter, trainer, num_epochs)

Step 0. time since epoch: 0.412. Train acc: 0.094. Train Loss: 2.314
Step 10. time since epoch: 3.955. Train acc: 0.095. Train Loss: 2.307
Step 20. time since epoch: 7.550. Train acc: 0.097. Train Loss: 2.306
Step 30. time since epoch: 11.085. Train acc: 0.098. Train Loss: 2.306
Step 40. time since epoch: 14.590. Train acc: 0.099. Train Loss: 2.305
Step 50. time since epoch: 18.152. Train acc: 0.106. Train Loss: 2.303
Step 60. time since epoch: 21.641. Train acc: 0.111. Train Loss: 2.302
Step 70. time since epoch: 25.143. Train acc: 0.117. Train Loss: 2.296
Step 80. time since epoch: 28.670. Train acc: 0.121. Train Loss: 2.280
Step 90. time since epoch: 32.588. Train acc: 0.129. Train Loss: 2.263
Step 100. time since epoch: 36.911. Train acc: 0.134. Train Loss: 2.250
Step 110. time since epoch: 41.000. Train acc: 0.141. Train Loss: 2.231
Step 120. time since epoch: 44.768. Train acc: 0.148. Train Loss: 2.210
Step 130. time since epoch: 48.327. Train acc: 0.155. Train Loss: 2.191
Step 1

## GoogleLeNet

<img src='https://drive.google.com/uc?id=1uus6Ytrlp-uhN_t5qQrzrG5y7R7BlnoE'>

In [ ]:
nn.Module

torch.nn.modules.module.Module

In [ ]:
class Inception(nn.Module):
    def __init__(self, ic, c1, c2, c3, c4, **kwargs):
        super(Inception, self).__init__(**kwargs)
        self.p1_1 = nn.Sequential(nn.Conv2d(ic, c1, kernel_size=1), nn.ReLU())
        self.p2_1 = nn.Sequential(nn.Conv2d(ic, c2[0], kernel_size=1), nn.ReLU())
        self.p2_2 = nn.Sequential(nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1), nn.ReLU())
        self.p3_1 = nn.Sequential(nn.Conv2d(ic, c3[0], kernel_size=1), nn.ReLU())
        self.p3_2 = nn.Sequential(nn.Conv2d(c3[0], c3[1], kernel_size=5, padding=2), nn.ReLU())
        self.p4_1 = nn.Sequential(nn.MaxPool2d(3, stride=1, padding=1))
        self.p4_2 = nn.Sequential(nn.Conv2d(ic, c4, kernel_size=1), nn.ReLU())

    def forward(self, x):
        p1 = self.p1_1(x)
        p2 = self.p2_2(self.p2_1(x))
        p3 = self.p3_2(self.p3_1(x))
        p4 = self.p4_2(self.p4_1(x))
        # Concatenate the outputs on the channel dimension.
        return torch.cat((p1, p2, p3, p4), dim=1)

<img src='https://drive.google.com/uc?id=1d8Fza-TWg4zRGQVIruPtgxbyniAQTPqB' width=400>

In [ ]:
b1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3), nn.ReLU(),
       nn.MaxPool2d(3, stride=2, padding=1))

In [ ]:
b2 = nn.Sequential(
       nn.Conv2d(64, 64, kernel_size=1),
       nn.Conv2d(64, 192, kernel_size=3, padding=1),
       nn.MaxPool2d(3, stride=2, padding=1))

In [ ]:
b3 = nn.Sequential(
       Inception(192, 64, (96, 128), (16, 32), 32),
       Inception(256, 128, (128, 192), (32, 96), 64),
       nn.MaxPool2d(3, stride=2, padding=1))

<img src='https://drive.google.com/uc?id=1UALYarkmGMEqThI7E15J7II850OVUkT7' width=400>

In [ ]:
b4 = nn.Sequential(
       Inception(480, 192, (96, 208), (16, 48), 64),
       Inception(512, 160, (112, 224), (24, 64), 64),
       Inception(512, 128, (128, 256), (24, 64), 64),
       Inception(512, 112, (144, 288), (32, 64), 64),
       Inception(528, 256, (160, 320), (32, 128), 128),
       nn.MaxPool2d(3, stride=2, padding=1))

In [ ]:
b5 = nn.Sequential(
       Inception(832, 256, (160, 320), (32, 128), 128),
       Inception(832, 384, (192, 384), (48, 128), 128),
       nn.AvgPool2d(7))

In [ ]:
net = nn.Sequential(b1, b2, b3, b4, b5, nn.Flatten(), nn.Linear(1024, 10))

In [ ]:
summary(net.to(device), input_size=(1, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           3,200
              ReLU-2         [-1, 64, 112, 112]               0
         MaxPool2d-3           [-1, 64, 56, 56]               0
            Conv2d-4           [-1, 64, 56, 56]           4,160
            Conv2d-5          [-1, 192, 56, 56]         110,784
         MaxPool2d-6          [-1, 192, 28, 28]               0
            Conv2d-7           [-1, 64, 28, 28]          12,352
              ReLU-8           [-1, 64, 28, 28]               0
            Conv2d-9           [-1, 96, 28, 28]          18,528
             ReLU-10           [-1, 96, 28, 28]               0
           Conv2d-11          [-1, 128, 28, 28]         110,720
             ReLU-12          [-1, 128, 28, 28]               0
           Conv2d-13           [-1, 16, 28, 28]           3,088
             ReLU-14           [-1, 16,

In [ ]:
lr, num_epochs = 0.001, 1
trainer = torch.optim.Adam(net.parameters(), lr=lr)
train(net, train_iter, test_iter, trainer, num_epochs)

Step 0. time since epoch: 0.946. Train acc: 0.094. Train Loss: 2.303
Step 10. time since epoch: 9.877. Train acc: 0.104. Train Loss: 2.302
Step 20. time since epoch: 18.779. Train acc: 0.103. Train Loss: 2.302
Step 30. time since epoch: 27.714. Train acc: 0.107. Train Loss: 2.302
Step 40. time since epoch: 36.668. Train acc: 0.118. Train Loss: 2.290
Step 50. time since epoch: 45.608. Train acc: 0.140. Train Loss: 2.251
Step 60. time since epoch: 54.543. Train acc: 0.160. Train Loss: 2.201
Step 70. time since epoch: 63.448. Train acc: 0.177. Train Loss: 2.148
Step 80. time since epoch: 72.326. Train acc: 0.192. Train Loss: 2.103
Step 90. time since epoch: 81.220. Train acc: 0.210. Train Loss: 2.055
Step 100. time since epoch: 90.101. Train acc: 0.228. Train Loss: 2.006
Step 110. time since epoch: 99.008. Train acc: 0.248. Train Loss: 1.957
Step 120. time since epoch: 107.900. Train acc: 0.265. Train Loss: 1.917
Step 130. time since epoch: 116.814. Train acc: 0.279. Train Loss: 1.885
Ste

## Предобученные архитектуры

In [ ]:
transoforms = tv.transforms.Compose([
    tv.transforms.Grayscale(3),
    tv.transforms.Resize((224, 224)),
    tv.transforms.ToTensor()
])

train_dataset = tv.datasets.MNIST('.', train=True, transform=transoforms, download=True)
test_dataset = tv.datasets.MNIST('.', train=False, transform=transoforms, download=True)
train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_iter = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

#### ResNet

In [ ]:
model = tv.models.resnet18(pretrained=True)

In [ ]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
summary(model.to(device), input_size=(3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

In [ ]:
# Убираем требование градиента:
for param in model.parameters():
    param.requires_grad = False

In [ ]:
model.fc

Linear(in_features=512, out_features=1000, bias=True)

In [ ]:
model.fc = nn.Linear(in_features=512, out_features=10)

In [ ]:
print("Params to learn:")
params_to_update = []
for name, param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

Params to learn:
	 fc.weight
	 fc.bias


In [ ]:
trainer = torch.optim.Adam(params_to_update, lr=0.001)

In [ ]:
train(model, train_iter, test_iter, trainer, 1)

Step 0. time since epoch: 1.024. Train acc: 0.148. Train Loss: 2.613
Step 10. time since epoch: 10.758. Train acc: 0.857. Train Loss: 0.506
Step 20. time since epoch: 20.596. Train acc: 0.916. Train Loss: 0.297
Step 30. time since epoch: 30.360. Train acc: 0.937. Train Loss: 0.222
Step 40. time since epoch: 40.174. Train acc: 0.949. Train Loss: 0.179
Step 50. time since epoch: 50.154. Train acc: 0.957. Train Loss: 0.152
Step 60. time since epoch: 59.808. Train acc: 0.962. Train Loss: 0.134
Step 70. time since epoch: 69.448. Train acc: 0.965. Train Loss: 0.121
Step 80. time since epoch: 79.040. Train acc: 0.968. Train Loss: 0.111
Step 90. time since epoch: 88.748. Train acc: 0.970. Train Loss: 0.102
Step 100. time since epoch: 98.313. Train acc: 0.973. Train Loss: 0.095
Step 110. time since epoch: 107.973. Train acc: 0.974. Train Loss: 0.090
Step 120. time since epoch: 117.594. Train acc: 0.975. Train Loss: 0.086
Step 130. time since epoch: 127.215. Train acc: 0.976. Train Loss: 0.082
S

#### DenseNet

In [ ]:
model = tv.models.densenet161(pretrained=True)

In [ ]:
model

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(96, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (rel

In [ ]:
# Убираем требование градиента:
for param in model.parameters():
    param.requires_grad = False

In [ ]:
model.classifier

Linear(in_features=2208, out_features=1000, bias=True)

In [ ]:
model.classifier = nn.Linear(in_features=2208, out_features=10)

In [ ]:
print("Params to learn:")
params_to_update = []
for name, param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

Params to learn:
	 classifier.weight
	 classifier.bias


In [ ]:
trainer = torch.optim.Adam(params_to_update, lr=0.001)

In [ ]:
train(model, train_iter, test_iter, trainer, 1)

Step 0. time since epoch: 2.244. Train acc: 0.086. Train Loss: 2.339
Step 10. time since epoch: 24.914. Train acc: 0.429. Train Loss: 1.986
Step 20. time since epoch: 47.535. Train acc: 0.586. Train Loss: 1.694
Step 30. time since epoch: 69.426. Train acc: 0.656. Train Loss: 1.490
Step 40. time since epoch: 91.260. Train acc: 0.696. Train Loss: 1.342
Step 50. time since epoch: 113.317. Train acc: 0.729. Train Loss: 1.221
Step 60. time since epoch: 135.461. Train acc: 0.751. Train Loss: 1.129
Step 70. time since epoch: 157.398. Train acc: 0.769. Train Loss: 1.054
Step 80. time since epoch: 179.258. Train acc: 0.785. Train Loss: 0.988
Step 90. time since epoch: 201.216. Train acc: 0.797. Train Loss: 0.933
Step 100. time since epoch: 223.297. Train acc: 0.808. Train Loss: 0.885
Step 110. time since epoch: 245.309. Train acc: 0.816. Train Loss: 0.846
Step 120. time since epoch: 267.280. Train acc: 0.823. Train Loss: 0.812
Step 130. time since epoch: 289.308. Train acc: 0.829. Train Loss: 0